# Effects of COVID-19 on CA hospital utilization trends

With the onset of COVID-19, hospitals statewide saw a sharp drop in inpatient discharges, emergency department utilization, and ambulatory surgeries.

In [30]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [150]:
import pandas as pd
import altair as alt
import altair_latimes as lat
import glob
import os

In [151]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get the data from the CDPH portal

In [178]:
url = "https://data.chhs.ca.gov/dataset/499e7fb6-459a-4ad4-ba00-490eeda20680/resource/7550f345-cb7d-4db6-b4a8-c0bb4f7bb09e/download/effects-of-covid-19-on-hospital-utilization-trends-statewide-montly-encounter-totals-by-category.csv"

In [179]:
src = pd.read_csv(url, parse_dates=["Date"])

In [180]:
src.columns = src.columns.str.lower()

In [181]:
src.head()

,date,setting,category,count
0,18-Jan,Ambulatory Surgery,Asthma,"11,746"
1,19-Jan,Ambulatory Surgery,Asthma,"12,747"
2,20-Jan,Ambulatory Surgery,Asthma,"12,248"
3,18-Feb,Ambulatory Surgery,Asthma,"11,085"
4,19-Feb,Ambulatory Surgery,Asthma,"11,741"


In [182]:
src.category.value_counts()

Asthma                                          96
Diabetes                                        96
Sepsis                                          96
Chronic Obstructive Pulmonary Disease (COPD)    96
Homeless                                        96
Cancer                                          96
Hypertension                                    96
Obesity                                         96
Stroke                                          96
COVID-19                                        24
Name: category, dtype: int64

In [183]:
src.setting.value_counts()

Ambulatory Surgery    306
Emergency Dept        306
Discharges            276
Name: setting, dtype: int64

In [184]:
src["date"] = pd.to_datetime(src["date"], format="%y-%b")

In [185]:
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month

In [186]:
src["count"] = src["count"].str.replace(",", "", regex=False).astype(int)

In [201]:
df = src[(src["setting"] != "Discharges")].copy()

---

In [202]:
emergency = df[df["setting"] == "Emergency Dept"]

In [203]:
surgery = df[df["setting"] == "Ambulatory Surgery"]

In [204]:
discharges = df[df["setting"] == "Discharges"]

--- 

In [212]:
cat_year = (
    df.groupby(["date", "category", "setting"]).agg({"count": "sum"}).reset_index()
)

In [213]:
alt.Chart(cat_year[cat_year["setting"] == "Emergency Dept"]).mark_area().encode(
    alt.X(
        "date:T", axis=alt.Axis(format="%B %Y", domain=False, grid=False, tickSize=0)
    ),
    alt.Y("count:Q"),
    alt.Color("category:N", scale=alt.Scale(scheme="tableau10")),
    facet=alt.Facet("setting", columns=1, title=None),
).properties(width=900, height=500).configure_view(strokeOpacity=0)

alt.Chart(...)

---

In [225]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9]
df[df["month"].isin(months)].groupby(["year", "category"]).agg(
    {"count": "sum"}
).reset_index()

,year,category,count
0,2018,Asthma,661695
1,2018,Cancer,216410
2,2018,Chronic Obstructive Pulmonary Disease (COPD),206643
3,2018,Diabetes,1184889
4,2018,Homeless,102914
5,2018,Hypertension,2312168
6,2018,Obesity,465545
7,2018,Sepsis,20789
8,2018,Stroke,12919
9,2019,Asthma,614321
